In [1]:
import numpy as np
import re
from pymol import cmd
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class NW():
    def __init__(
            self, seqa:str, seqb:str, 
            match, mismatch, gap
    ):
        self.seqa = seqa
        self.seqb = seqb
        self.match_score = match
        self.mismatch_score = mismatch
        self.gap_score = gap
        self.matrix = np.zeros((len(seqa)+1, len(seqb)+1))
        self.matrix.fill(np.inf)
        self.matrix[:, 0] = [self.gap_score * i for i in range(self.matrix.shape[0])]
        self.matrix[0, :] = [self.gap_score * j for j in range(self.matrix.shape[1])]

                
    def _scoring(self):
        for row_idx, i in enumerate(self.matrix):
            for col_idx, j in enumerate(i):
                
                if j != np.inf:
                    pass
                else:
                    '''
                    lt: right top (diagonal)
                    lt_: rt + match or mismatch
                    rb: right bottom
                    lt: left top
                    '''
                    lt = self.matrix[row_idx-1, col_idx-1]
                    lt_ = lt + self.match_score if self.seqa[row_idx-1] == self.seqb[col_idx-1] else lt + self.mismatch_score
                    
                    lb = self.matrix[row_idx, col_idx-1]
                    rt = self.matrix[row_idx-1, col_idx]

                    score = np.max([lt_, lb+self.gap_score, rt+self.gap_score])

                    self.matrix[row_idx, col_idx] = score

        return self.matrix
    
    def _traceback(self,):
        seqa_output, seqb_output = [], []
        row = self.matrix.shape[0] - 1
        col = self.matrix.shape[1] - 1

        while row > 0 or col > 0:

            # How N&W gets rid of local optimum

            #print(f'row: {row}, col: {col}')
            current_score = self.matrix[row, col]

            if row > 0 and col >0:
                #match_val = self.match_score if self.seqa[row-1] == self.seqb[col-1] else self.mismatch_score
                match_val = self.match_score if self.seqa[row-1] == self.seqb[col-1] else self.mismatch_score
                #print(match_val)
                if current_score == self.matrix[row-1, col - 1] + match_val:
                    seqa_output.append(self.seqa[row-1])
                    seqb_output.append(self.seqb[col-1])
                    
                    row -= 1
                    col -= 1
                    continue
                
            if col>0 and current_score == self.matrix[row, col - 1] + self.gap_score:
                seqa_output.append('-')
                seqb_output.append(self.seqb[col-1])
                col -= 1
                continue

            if row > 0 and current_score == self.matrix[row - 1, col] + self.gap_score:
                
                seqa_output.append(self.seqa[row-1])
                seqb_output.append('-')
                row -= 1
                
                continue

                
        seqa_output.reverse(), seqb_output.reverse()
        return seqa_output, seqb_output
    
    def align(self):
        score_matrix = self._scoring()
        #print(score_matrix)
        seqa_output, seqb_output = self._traceback()
        return seqa_output, seqb_output

In [3]:
try:
    cmd.load('cox1.pdb')
    cmd.load('cox2.pdb')
    print('Load models from local files.')
except:
    cmd.fetch('6y3c')
    cmd.fetch('5ikr')
    cmd.set_name('5ikr', 'cox2')
    cmd.set_name('6y3c', 'cox1')
    print(f'Chain labels on Cox2{cmd.get_chains('cox2')}')
    cmd.remove('organic')
    cmd.remove('inorganic')
    cmd.remove('solvent')
    cmd.remove('cox2 and chain B')
    cmd.remove('hydrogens')
    print(f'Chain labels on Cox2 (monomer){cmd.get_chains('cox2')}')
    print(f'Chain labels on Cox1{cmd.get_chains('cox1')}')
    cmd.save('cox1.pdb', 'cox1')
    cmd.save('cox2.pdb', 'cox2')
finally:
    c1 = cmd.get_model('cox1')
    c2 = cmd.get_model('cox2')

Load models from local files.


In [ ]:
class lDDT():


    def __init__(self, objects:list, obj_names:list, cutoff:int|float=15.0, thresholds:list=[0.5, 1.0, 2.0, 4.0]):
        self.all_objects = objects
        self.all_names = obj_names
        for i, o in enumerate(objects):
            obj_name = f'obj{i}'
            setattr(self, obj_name , o)

        self.cutoff = cutoff
        self.thresholds = thresholds


    @property
    def fasta(self):
        all_fasta = []
        for name in self.all_names:
            obj_fasta = cmd.get_fastastr(f'{name}')
            obj_fasta = re.findall(r'\n(\w*)', obj_fasta)
            seperator = ''
            obj_fasta = seperator.join(obj_fasta)
            all_fasta.append(obj_fasta)
        return all_fasta
    

    def read_fasta(self):
        with open('F:/coxpdb.txt', 'r') as f:
            content = f.read()
            sequences = re.findall('>.*\n(.*)', content)

        seq1 = ''
        seq2 = ''
        for i in range(len(sequences[0])):
            if sequences[0][i] != '-' and sequences[1][i] !='-':
                seq1 += sequences[0][i]
                seq2 += sequences[1][i]

        return seq1, seq2


    def sequence_alignment(self, method='NW', seq_mask = False,match=1, mismatch=-1, gap=-2):
        if method == 'NW':
            nw = NW(*self.fasta,match=match, mismatch=mismatch, gap=gap)
            seq1_,seq2_ = nw.align()
            seperator = ''
            seq1_ = seperator.join(seq1_)
            seq2_ = seperator.join(seq2_)
            if seq_mask:
                seq1_, seq2_ = self._mask_seq(seq1_, seq2_)
            return seq1_, seq2_


    def _mask_seq(self, seq1, seq2):
        if len(seq1) != len(seq2):
            raise ValueError('seq1 and seq2 must have the same length')

        residues1 = list(dict.fromkeys([a.resi for a in self.all_objects[0].atom]))
        residues2 = list(dict.fromkeys([a.resi for a in self.all_objects[1].atom]))

        seq_mask1, seq_mask2 = [], []
        res_idx1, res_idx2 = 0, 0  

        for i in range(len(seq1)):
            aa1, aa2 = seq1[i], seq2[i]

            if aa1 != '-':
                res1 = residues1[res_idx1]
                res_idx1 += 1
            else:
                res1 = None

            if aa2 != '-':
                res2 = residues2[res_idx2]
                res_idx2 += 1
            else:
                res2 = None

            if aa1 != '-' and aa2 != '-':
                seq_mask1.append(res1)
                seq_mask2.append(res2)
        
        return seq_mask1, seq_mask2


    def get_coords(self, resi1, resi2):
        cmd.select(f'mask_{self.all_names[0]}', f'{self.all_names[0]} and resi {'+'.join(resi1)} and name CA')
        cmd.select(f'mask_{self.all_names[1]}', f'{self.all_names[1]} and resi {'+'.join(resi2)} and name CA')
        cmd.align(f'mask_{self.all_names[0]}', f'mask_{self.all_names[1]}')
        seq1_coord = cmd.get_coords(f'mask_{self.all_names[0]}')
        seq2_coord = cmd.get_coords(f'mask_{self.all_names[1]}')
        return seq1_coord, seq2_coord
    

    def lddt_masked(self, coord1, coord2, cutoff=15.0, thresholds=[0.5, 1.0, 2.0, 4.0]):

        coord1 = torch.tensor(coord1, device=device)
        coord2 = torch.tensor(coord2, device=device)

        dist_obj1 = torch.cdist(coord1, coord1)
        dist_obj2 = torch.cdist(coord2, coord2)

        mask = (dist_obj1 < cutoff) & (dist_obj1 > 0)
        diff = torch.abs(dist_obj1 - dist_obj2)

        scores = sum((diff < t) & mask for t in thresholds) / len(thresholds)
        scores = scores.sum(dim=-1) / mask.sum(dim=-1).clamp(min=1)
        return scores.mean()    

l = lDDT([c1, c2], ['cox1', 'cox2'])
l.fasta

['GAPTPVNPCCYYPCQHQGICVRFGLDRYQCDCTRTGYSGPNCTIPGLWTWLRNSLRPSPSFTHFLLTHGRWFWEFVNATFIREMLMRLVLTVRSNLIPSPPTYNSAHDYISWESFSNVSYYTRILPSVPKDCPTPMGTKGKKQLPDAQLLARRFLLRRKFIPDPQGTNLMFAFFAQHFTHQFFKTSGKMGPGFTKALGHGVDLGHIYGDNLERQYQLRLFKDGKLKYQVLDGEMYPPSVEEAPVLMHYPRGIPPQSQMAVGQEVFGLLPGLMLYATLWLREHNRVCDLLKAEHPTWGDEQLFQTTRLILIGETIKIVIEEYVQQLSGYFLQLKFDPELLFGVQFQYRNRIAMEFNHLYHWHPLMPDSFKVGSQEYSYEQFLFNTSMLVDYGVEALVDAFSRQIAGRIGGGRNMDHHILHVAVDVIRESREMRLQPFNEYRKRFGMKPYTSFQELVGEKEMAAELEELYGDIDALEFYPGLLLEKCHPNSIFGESMIEIGAPFSLKGLLGNPICSPEYWKPSTFGGEVGFNIVKTATLKKLVCLNTKTCPYVSFRVPD',
 'NPCCSHPCQNRGVCMSVGFDQYKCDCTRTGFYGENCSTPEFLTRIKLFLKPTPNTVHYILTHFKGFWNVVNNPFLRNAIMSYVLTSRSHLIDSPPTYNADYGYKSWEAFSNLSYYTRALPPVPDDCPTPLGVKGKKQLPDSNEIVEKLLLRRKFIPDPQGSNMMFAFFAQHFTHQFFKTDHKRGPAFTNGLGHGVDLNHIYGETLARQRKLRLFKDGKMKYQIIDGEMYPPTVKDTQAEMIYPPQVPEHLRFAVGQEVFGLVPGLMMYATIWLREHNRVCDVLKQEHPEWGDEQLFQTSRLILIGETIKIVIEDYVQHLSGYHFKLKFDPELLFNKQFQYQNRIAAEFNTLYHWHPLLPDTFQIHDQKYNYQQFIYNNSILLEHGITQFVESFTRQIAGRVAGGRNVPPAVQKVSQASIDQSRQMKYQSFNEYRKR

In [20]:
a, b = l.sequence_alignment(seq_mask=True)
seq1_coord, seq2_coord = l.get_coords(a, b)

In [21]:
l.lddt_masked(seq1_coord, seq2_coord)

tensor(0.9195, device='cuda:0')